# Stereotype classification

## Inhaltsverzeichnis
...

## Einführung (Kalie)
...

## Daten (Kalie)
...

## Preselection & Annotation (Kalie)
...

In [216]:
import spacy
import numpy as np
import pandas as pd
from math import log
from textblob_de import TextBlobDE
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.dummy import DummyClassifier

In [162]:
pd.set_option('display.max_colwidth', None)

In [163]:
nlp = spacy.load('de_core_news_sm')
de_stopwords = stopwords.words('german')

## Preprocessing (Kalie)
...

In [164]:
def preprocess(sent):
    # Tokenize
    doc = nlp(sent)
    tokens = [token.text for token in doc]
    # To lower
    tokens_lower = [t.lower() for t in tokens]
    # Remove punctuation and stopwords
    clean = [t for t in tokens_lower if t.isalpha() and t not in de_stopwords]
    return " ".join(clean)


def apply_preprocessing(sents):
    clean_sents = []
    for sent in sents:
        clean_sents.append(preprocess(sent))
    return np.array(clean_sents)

In [165]:
# Load annotated sentences
raw_df = pd.read_csv("data/combined_samples.csv", sep="\t")
raw_df.drop("Unnamed: 0", axis=1, inplace=True)
raw_df

,Term_NDoc_Date,Sentence,Frame
0,18_100_2015-04-23,Erste Beratung des von der Bundesregierung eingebrachten Entwurfs eines Gesetzes zur Förderung von Investitionen finanzschwacher Kommunen und zur Entlastung von Ländern und Kommunen bei der Aufnahme und Unterbringung von Asylbewerbern Drucksache 18/4653 (neu) ............,x
1,18_100_2015-04-23,Mit dem vorliegenden Entwurf eines Nachtragshaushalts 2015 und dem Entwurf eines Gesetzes zur Förderung von Investitionen finanzschwacher Kommunen und zur Entlastung von Ländern und Kommunen bei der Aufnahme und Unterbringung von Asylbewerbern setzt die Bundesregierung ihre Politik für eine größere Tragfähigkeit der öffentlichen Finanzen und für Wachstum fort.,3
2,18_100_2015-04-23,"Berlin, Donnerstag, den 23. April 2015 9485 Dr. Dietmar Bartsch (A) (C) (D)(B) bereitstellen – auch in Kenntnis der Tatsache, dass die Übergriffe auf Ausländer, auf Unterkünfte der Asylbewerber zugenommen haben und dass Menschen, die im Ehrenamt diese Willkommenskultur, von der wir alle sprechen, leben, eben nicht wenige Probleme haben.",2.1
3,18_100_2015-04-23,"Es ist überhaupt nicht hinnehmbar, wenn der Generalsekretär einer immerhin regierungstragenden Partei, der CSU, sagt, das Asylrecht sei nicht für Sozialtouristen gemacht, die einen Freifahrtschein ins „All-inclusive-Sozialparadies“ buchen wollen.",4.4
4,18_100_2015-04-23,"Denn auch unter dem Gesichtspunkt – Kollege Bartsch hat auf die Flüchtlingsströme hingewiesen –, dass wir in den nächsten Jahren große Anstrengungen unternehmen müssen, um Flüchtlinge aufzunehmen und zu integrieren – Integration ist fast genauso wichtig –, muss die Leistungsbereitschaft der Kommunen gewährleistet sein.",4.1
...,...,...,...
495,18_121_2015-09-10,"Das sind die drei Integrationsaufgaben, die wir haben.",x
496,18_121_2015-09-10,Ich komme jetzt auch zum Thema Fluchtursachenbekämpfung.,x
497,18_121_2015-09-10,"Dass viele Mitbürger in unserem Land angesichts einer solchen Anzahl von Flüchtlingen Ängste haben und sich fragen, wie Bund, Länder und Kommunen die Unterbringung und auch die Integration bewältigen wollen, ist, wie ich glaube, nur allzu verständlich.",4
498,18_121_2015-09-10,"Denn nur wenn es gelingt, gemeinsam mit der Wirtschaft das Potenzial und die Motivation der Menschen, die zu uns kommen, zu nutzen, nur dann wird Integration gelingen.",3


In [166]:
raw_sents = raw_df["Sentence"].to_numpy()
# Preprocess
sents = apply_preprocessing(raw_sents)
sents.size

500

## Vorüberlegungen & Ziele (Liane)
Sanchez et al. untersuchen in ihrem Paper, wie effektiv klassische ML-Modelle und SOA-Modelle mithilfe der erstellten Taxonomie und des darauf basierenden annotierten Korpus das Framing von Migrant\*innen klassifizieren können. Dabei werden zwei binäre Klassifikationsprobleme betrachtet:
- Enthält eine Aussage Stereotype über Migrant\*innen oder nicht? ("Stereotype vs. Nonstereotype")
- Angenommen, die Aussage enthält Stereotype: Werden Migrant\*innen als Opfer oder Bedrohung dargestellt? ("Victim vs. Threat")

Wir führen diese Aufgaben mit einem klassischen ML-Modell für überwachtes Lernen, der Support Vector Machine, durch. Wir nutzen, wie die Autor\*innen des Papers, die Implementation von Scikit-learn. Sanchez et al. trainieren die Modelle mit der tfidf-gewichteten Bag-of-words Repräsentation der Sätze. Es werden Unigramme, Bigramme und Trigramme verwendet, wobei Unigramme die besten Ergebnisse erzielen.
Im Rahmen dieses Projektes nutzen wir ebenfalls die Bag-of-words Repräsentation, wobei wir das Modell sowohl mit tfidf-Gewichtung als auch ohne tfidf-Gewichtung trainieren. Außerdem testen wir Sentiment als Feature. (Weitere Erklärungen zu ergänzen...)

In [167]:
# Array with original indexes so we can retrieve raw sentences later
orig_indexes = np.arange(raw_sents.size)
orig_indexes.size

500

## 1. Aufgabe: Stereotype vs. Keine Stereotype
Die Annotator\*innen wurden gebeten, die Aussagen über Migrant\*innen Frames zuzuordnen. Sanchez et al. erheben den Anspruch, dass die Taxonomie von Frames, vollständig die Dimensionen der Stereotypisierung von Migrant\*innen abbildet, sei sie positiv oder negativ.
Sätze, welche keinem Frame oder keiner Kategorie zugeordnet werden konnten, sollten mit "x" annotiert werden. 
Dem Datensatz wird eine Spalte "Contains stereotypes" mit den Goldlabels für die Klassifikation hinzugefügt. Sätze, welche mit "x" annotiert wurden, erhalten das Label 0 für "enthält keine Stereotype". Alle anderen Sätze erhalten 1 als Label.

In [168]:
def get_binary_labels(data):
    """Add column to df with binary labels for classification"""
    data.loc[data["Frame"] == "x", "Contains stereotypes"] = 0
    data.loc[data["Frame"] != "x", "Contains stereotypes"] = 1
    return data

In [169]:
df = get_binary_labels(raw_df)
df.head()

,Term_NDoc_Date,Sentence,Frame,Contains stereotypes
0,18_100_2015-04-23,Erste Beratung des von der Bundesregierung eingebrachten Entwurfs eines Gesetzes zur Förderung von Investitionen finanzschwacher Kommunen und zur Entlastung von Ländern und Kommunen bei der Aufnahme und Unterbringung von Asylbewerbern Drucksache 18/4653 (neu) ............,x,0.0
1,18_100_2015-04-23,Mit dem vorliegenden Entwurf eines Nachtragshaushalts 2015 und dem Entwurf eines Gesetzes zur Förderung von Investitionen finanzschwacher Kommunen und zur Entlastung von Ländern und Kommunen bei der Aufnahme und Unterbringung von Asylbewerbern setzt die Bundesregierung ihre Politik für eine größere Tragfähigkeit der öffentlichen Finanzen und für Wachstum fort.,3,1.0
2,18_100_2015-04-23,"Berlin, Donnerstag, den 23. April 2015 9485 Dr. Dietmar Bartsch (A) (C) (D)(B) bereitstellen – auch in Kenntnis der Tatsache, dass die Übergriffe auf Ausländer, auf Unterkünfte der Asylbewerber zugenommen haben und dass Menschen, die im Ehrenamt diese Willkommenskultur, von der wir alle sprechen, leben, eben nicht wenige Probleme haben.",2.1,1.0
3,18_100_2015-04-23,"Es ist überhaupt nicht hinnehmbar, wenn der Generalsekretär einer immerhin regierungstragenden Partei, der CSU, sagt, das Asylrecht sei nicht für Sozialtouristen gemacht, die einen Freifahrtschein ins „All-inclusive-Sozialparadies“ buchen wollen.",4.4,1.0
4,18_100_2015-04-23,"Denn auch unter dem Gesichtspunkt – Kollege Bartsch hat auf die Flüchtlingsströme hingewiesen –, dass wir in den nächsten Jahren große Anstrengungen unternehmen müssen, um Flüchtlinge aufzunehmen und zu integrieren – Integration ist fast genauso wichtig –, muss die Leistungsbereitschaft der Kommunen gewährleistet sein.",4.1,1.0


In [170]:
# Get goldlabels
y = df["Contains stereotypes"].to_numpy()

### Extraktion von Features für überwachtes Lernen
#### 1. Bag-of-Ngrams ohne Tfidf-Gewichtung
Zunächst wandeln wir den Datensatz in seine Bag-of-words bwz. "Bag-of-ngrams" Repräsentation um. Jeder Satz wird also als Vektor dargestellt, dessen Werte die Counts aller im Datensatz vorkommenden Ngramme darstellen.

In [171]:
def get_ngram_count_matrx(sents, n):
    """Get ngrams and ngram count matrix"""
    vectorizer = CountVectorizer(ngram_range=(n, n))
    ngram_count_matrx = vectorizer.fit_transform(sents).toarray()
    ngrams = vectorizer.get_feature_names_out()
    return ngrams, ngram_count_matrx

In [172]:
unigrams, unigram_count_matrx = get_ngram_count_matrx(sents, 1)
bigrams, bigram_count_matrx = get_ngram_count_matrx(sents, 2)
trigrams, trigram_count_matrx = get_ngram_count_matrx(sents, 3)

In [173]:
# Number of unique unigrams, bigrams and trigrams
unigrams.size, bigrams.size, trigrams.size

(2457, 4371, 4091)

In [174]:
unigram_count_matrx.shape, bigram_count_matrx.shape, trigram_count_matrx.shape

((500, 2457), (500, 4371), (500, 4091))

#### 2. Bag-of-ngrams mit Tfidf-Gewichtung
Nun ziehen wir das Tfidf-Maß hinzu, um die Counts der Ngramme zu gewichten.

In [175]:
# Add weighing with tfidf
def transform_to_tfidf(ngram_count_matrx):
    transformer = TfidfTransformer()
    tfidf_matrx = transformer.fit_transform(ngram_count_matrx).toarray()
    return tfidf_matrx

In [176]:
unigram_tfidf_matrx = transform_to_tfidf(unigram_count_matrx)
bigram_tfidf_matrx = transform_to_tfidf(bigram_count_matrx)
trigram_tfidf_matrx = transform_to_tfidf(trigram_count_matrx)

In [177]:
unigram_tfidf_matrx.shape, bigram_tfidf_matrx.shape, trigram_tfidf_matrx.shape

((500, 2457), (500, 4371), (500, 4091))

#### 3. Sentiment Scores
Als letztes berechnen wir für jeden Satz noch den Sentiment score. Dafür nutzen wir das Package TextBlobDE.

In [178]:
def get_sentiment(sents):
    sentiment = []
    for sent in sents:
        blob = TextBlobDE(sent)
        sentiment.append(blob.polarity)
    return np.array(sentiment)

In [179]:
sentiment_scores = get_sentiment(sents).reshape(-1, 1)
sentiment_scores.shape

(500, 1)

### Trainieren des Modells

In [180]:
def train_and_evaluate_model(X, y, orig_indexes, model):
    # Split into training and test data
    X_train, X_test, y_train, y_test, orig_train, orig_test = train_test_split(
    X, y, orig_indexes,
    test_size=0.2, 
    stratify=y, 
    random_state=10
    )
    # Fit model
    model.fit(X_train, y_train)
    # Make predictions
    predicted = model.predict(X_test)
    # Create dataframe with raw sents of test set and predictions from model
    predictions = []
    for i in range(len(predicted)):
        predictions.append([raw_sents[orig_test[i]], predicted[i], y_test[i]])
    predictions_df = pd.DataFrame(predictions, columns=["Raw sentence", "Predicted label", "Gold label"])
    # Evaluate model
    results = classification_report(y_test, predicted)
    
    return predictions_df, results

In [181]:
def get_correct(predictions):
    return predictions.loc[predictions["Predicted label"] == predictions["Gold label"]]

def get_wrong(predictions):
    return predictions.loc[predictions["Predicted label"] != predictions["Gold label"]]

#### 1.1. Unigramm-Counts

In [182]:
# Train Support Vector Machine with unigram counts
predictions, results = train_and_evaluate_model(unigram_count_matrx, y, orig_indexes, SVC())
print(results)

              precision    recall  f1-score   support

         0.0       0.62      0.93      0.75        60
         1.0       0.60      0.15      0.24        40

    accuracy                           0.62       100
   macro avg       0.61      0.54      0.49       100
weighted avg       0.61      0.62      0.54       100



#### 1.2. Bigramm-Counts

In [183]:
# Train Support Vector Machine with bigram counts
predictions, results = train_and_evaluate_model(bigram_count_matrx, y, orig_indexes, SVC())
print(results)

              precision    recall  f1-score   support

         0.0       0.61      0.95      0.74        60
         1.0       0.50      0.07      0.13        40

    accuracy                           0.60       100
   macro avg       0.55      0.51      0.44       100
weighted avg       0.56      0.60      0.50       100



#### 1.3. Trigramm-Counts

In [184]:
# Train Support Vector Machine with trigram counts
predictions, results = train_and_evaluate_model(trigram_count_matrx, y, orig_indexes, SVC())
print(results)

              precision    recall  f1-score   support

         0.0       0.61      0.97      0.75        60
         1.0       0.60      0.07      0.13        40

    accuracy                           0.61       100
   macro avg       0.61      0.52      0.44       100
weighted avg       0.61      0.61      0.50       100



#### 2.1. Unigramm-Counts mit Tfidf-Gewichtung

In [185]:
# Train Support Vector Machine with unigram tfidf
predictions, results = train_and_evaluate_model(unigram_tfidf_matrx, y, orig_indexes, SVC())
print(results)

              precision    recall  f1-score   support

         0.0       0.61      1.00      0.76        60
         1.0       1.00      0.05      0.10        40

    accuracy                           0.62       100
   macro avg       0.81      0.53      0.43       100
weighted avg       0.77      0.62      0.49       100



#### 2.2 Bigramm-Counts mit Tfidf-Gewichtung

In [186]:
# Train Support Vector Machine with bigram tfidf
predictions, results = train_and_evaluate_model(bigram_tfidf_matrx, y, orig_indexes, SVC())
print(results)

              precision    recall  f1-score   support

         0.0       0.60      1.00      0.75        60
         1.0       0.00      0.00      0.00        40

    accuracy                           0.60       100
   macro avg       0.30      0.50      0.37       100
weighted avg       0.36      0.60      0.45       100



/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

#### 2.3. Trigramm-Counts mit Tfidf-Gewichtung

In [187]:
# Train Support Vector Machine with trigram tfidf
predictions, results = train_and_evaluate_model(trigram_tfidf_matrx, y, orig_indexes, SVC())
print(results)

              precision    recall  f1-score   support

         0.0       0.60      1.00      0.75        60
         1.0       0.00      0.00      0.00        40

    accuracy                           0.60       100
   macro avg       0.30      0.50      0.37       100
weighted avg       0.36      0.60      0.45       100



/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

### 3. Sentiment scores

In [188]:
# Train Support Vector Machine with sentiment scores
predictions, results = train_and_evaluate_model(sentiment_scores, y, orig_indexes, SVC())
print(results)

              precision    recall  f1-score   support

         0.0       0.60      1.00      0.75        60
         1.0       0.00      0.00      0.00        40

    accuracy                           0.60       100
   macro avg       0.30      0.50      0.37       100
weighted avg       0.36      0.60      0.45       100



/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

### 4. Kombinationen von Features
Wir testen außerdem, ob eine Kombinationen von unterschiedlichen Features eine Verbesserung der Performance bewirkt.

In [189]:
def combine_features(*features):
    return np.concatenate(features, axis=1)

In [190]:
unigrams_sentiment = combine_features(unigram_count_matrx, sentiment_scores)
bigrams_sentiment = combine_features(bigram_count_matrx, sentiment_scores)
trigrams_sentiment = combine_features(trigram_count_matrx, sentiment_scores)

#### 4.1. Unigramm-Counts + Sentiment

In [191]:
# Train Support Vector Machine with unigrams and sentiment scores
predictions, results = train_and_evaluate_model(unigrams_sentiment, y, orig_indexes, SVC())
print(results)

              precision    recall  f1-score   support

         0.0       0.63      0.95      0.75        60
         1.0       0.67      0.15      0.24        40

    accuracy                           0.63       100
   macro avg       0.65      0.55      0.50       100
weighted avg       0.64      0.63      0.55       100



#### 4.2. Bigramm-Counts + Sentiment

In [192]:
# Train Support Vector Machine with bigrams and sentiment scores
predictions, results = train_and_evaluate_model(bigrams_sentiment, y, orig_indexes, SVC())
print(results)

              precision    recall  f1-score   support

         0.0       0.61      0.97      0.75        60
         1.0       0.60      0.07      0.13        40

    accuracy                           0.61       100
   macro avg       0.61      0.52      0.44       100
weighted avg       0.61      0.61      0.50       100



#### 4.3. Trigramm-Counts + Sentiment

In [193]:
# Train Support Vector Machine with trigrams and sentiment scores
predictions, results = train_and_evaluate_model(trigrams_sentiment, y, orig_indexes, SVC())
print(results)

              precision    recall  f1-score   support

         0.0       0.61      0.97      0.75        60
         1.0       0.60      0.07      0.13        40

    accuracy                           0.61       100
   macro avg       0.61      0.52      0.44       100
weighted avg       0.61      0.61      0.50       100



In [195]:
unigrams_tfidf_sentiment = combine_features(unigram_tfidf_matrx, sentiment_scores)
bigrams_tfidf_sentiment = combine_features(bigram_tfidf_matrx, sentiment_scores)
trigrams_tfidf_sentiment = combine_features(trigram_tfidf_matrx, sentiment_scores)

#### 4.4. Unigramm-Tfidf + Sentiment

In [196]:
# Train Support Vector Machine with unigram tfidf and sentiment scores
predictions, results = train_and_evaluate_model(unigrams_tfidf_sentiment, y, orig_indexes, SVC())
print(results)

              precision    recall  f1-score   support

         0.0       0.61      1.00      0.75        60
         1.0       1.00      0.03      0.05        40

    accuracy                           0.61       100
   macro avg       0.80      0.51      0.40       100
weighted avg       0.76      0.61      0.47       100



#### 4.5. Bigramm-Tfidf + Sentiment

In [197]:
# Train Support Vector Machine with bigram tfidf and sentiment scores
predictions, results = train_and_evaluate_model(bigrams_tfidf_sentiment, y, orig_indexes, SVC())
print(results)

              precision    recall  f1-score   support

         0.0       0.60      1.00      0.75        60
         1.0       0.00      0.00      0.00        40

    accuracy                           0.60       100
   macro avg       0.30      0.50      0.37       100
weighted avg       0.36      0.60      0.45       100



/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

#### 4.6. Trigramm-Tfidf + Sentiment

In [198]:
# Train Support Vector Machine with trigram tfidf and sentiment scores
predictions, results = train_and_evaluate_model(trigrams_tfidf_sentiment, y, orig_indexes, SVC())
print(results)

              precision    recall  f1-score   support

         0.0       0.60      1.00      0.75        60
         1.0       0.00      0.00      0.00        40

    accuracy                           0.60       100
   macro avg       0.30      0.50      0.37       100
weighted avg       0.36      0.60      0.45       100



/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

### Besten Ergebnisse genauer betrachten:

In [199]:
# Best features: unigram counts + sentiment
predictions, results = train_and_evaluate_model(unigrams_sentiment, y, orig_indexes, SVC())

In [200]:
# Correct predictions
get_correct(predictions)

,Raw sentence,Predicted label,Gold label
1,"Ich nehme kein Beispiel aus dem Bereich des Terrorismus, der Kinderpornografie und auch nicht der Vergewaltigung, sondern ich nehme ein sehr praktisches Beispiel.",0.0,0.0
2,"Da die Zuwanderer solche familiären Netzwerke hier nicht haben, müssen sie sich auch diesbezüglich neu orientierten.",0.0,0.0
5,"Den Bundesinnenminister als denjenigen zu bezeichnen, der der Integration im Wege stand, heißt wirklich, das, was die Union in den letzten Jahren mit dieser Regierung getan hat, vollkommen zu verleugnen.",0.0,0.0
6,"Wir unterscheiden zwischen jenen, die wegen Krieg und Verfolgung Aussicht auf Asyl haben, und denen, die keine Chance auf eine Zukunft in Deutschland haben.",0.0,0.0
8,Innerhalb von acht Wochen wurden allein 28 000 Asylanträge von Menschen aus dem Kosovo gestellt.,0.0,0.0
...,...,...,...
92,"Dem gleichen Ziel dient die Aufforderung der Financial Action Task Force der OECD, jegliche Terrorismusfinanzierung unter Strafe zu stellen.",0.0,0.0
94,"Habe ich überhaupt eine Chance auf Asyl, wenn ich mein Leben aufs Spiel setze, wenn ich meine Familie, wenn ich meine Kinder verlasse?",0.0,0.0
95,"Auch wenn wir den Bürgerinnen und Bürgern ein Gefühl dafür geben wollen, dass wir mit der außenpolitischen Aufgabe der Bekämpfung der Fluchtursachen verantwortungsvoll umgehen, werden wir ihnen natürlich nicht für jedes Problem eine Lösung servieren können.",0.0,0.0
96,Eine ähnliche Schnittstellenfunktion erfüllt der Generalbundesanwalt beim Thema islamistischer Terrorismus.,0.0,0.0


In [201]:
# Wrong predictions
get_wrong(predictions)

,Raw sentence,Predicted label,Gold label
0,"Solange Menschen keine andere Chance sehen, als über Schlepper und Menschenhändler nach Europa zu kommen, werden wir das Elend an unseren Grenzen nicht los.",0.0,1.0
3,"Aber während wir über unsere Willkommenskultur sprechen, brennen Unterkünfte für Flüchtlinge, werden Anschläge verübt.",0.0,1.0
4,"Nur wenn wir auch in den Herkunftsländern Fluchtursachen bekämpfen und wenn wir Fehlanreize in Deutschland beseitigen, dann werden wir unsere Kommunen dauerhaft und nachhaltig entlasten.",0.0,1.0
7,"Denn wie wollen Sie bitte schön erklären, dass man Syrer nicht nach Ungarn, Italien oder Bulgarien abschieben darf, eritreische oder irakische Flüchtlinge aber schon?",0.0,1.0
9,Einwanderung ist essenziell erforderlich für unsere Gesellschaft.,0.0,1.0
10,"Aber wenn wir das ernst meinen – wir meinen es alle ernst –, dass die Fluchtursachen langfristig nur in den Krisenregionen bekämpft werden können, dann müssen wir uns auch dazu bekennen, dass alle Etats in unserem Haushalt, die sich um die internationale Verantwortung kümmern – humanitäre Hilfe, Polizeiausbildung, wirtschaftliche Zusammenarbeit, aber auch der Etat für Verteidigung –, in den nächsten Jahren deutlicher anwachsen müssen, um dieser Verantwortung gerecht zu werden.",1.0,0.0
11,Damit können und sollen die Fluchtursachen in den wichtigsten Herkunftsländern zusätzlich bekämpft werden.,0.0,1.0
22,"Erst die wirtschaftliche Leistungskraft unseres Landes, gepaart mit soliden Finanzen, erspart uns jetzt schwere Entscheidungen und Konflikte darüber, wie wir die Aufnahme und die Integration so vieler Menschen in Deutschland schaffen und finanzieren wollen.",1.0,0.0
25,Über das novellierte Asylbewerberleistungsgesetz übernimmt der Bund jährlich Kosten von 43 Millionen Euro.,0.0,1.0
29,"Ich bin überzeugt, dass wir beides brauchen: legale Möglichkeiten der Einwanderung und die Verfolgung von gewissenlosen Schleppern.",0.0,1.0


## Majority Baseline

In [219]:
# Train baseline
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(sents, y)
dummy_clf.score(sents,y)

0.604

### Interpretation:
...

## Assoziation zwischen Ngrammen und Labels
Sanchez et al. argumentieren, dass Stereotype nicht nur in Form von Zuschreibungen negativer Attribute einer Personengruppe auftreten, sondern oft deutlich subtiler sind. Vielmehr zeichnen sich Stereotype durch das Framing, also durch das Narrativ und den Kontext, in welches die jeweilige Personengruppe platziert wird, aus. Indem das Framing von Migration/Migrant\*innen annotiert wird, sollen auch diejenigen Aussagen über Migrant\*innen, welche nicht auf den ersten Blick als stereotypisch erkennbar sind, identifiziert werden. 

Die Autor\*innen des Papers kommen auf einen interessanten Ansatz, in dem die Bereicherung der Sozialwissenschaften durch komputationelle Verfahren sichtbar wird. Sie berechnen für alle Ngramme (Bi- und Trigramme) den **PMI (Pointwise Mutual Information)** zu den jeweiligen Labels (also "Stereotype vs. Nonstereotype" bzw. "Victims vs. Threats"). PMI ist ein statistisches Maß, welches die Assoziation zwischen Ereignissen misst. Mit anderen Worten:  Zum einen untersuchen die Autor\*innen, welche Ngramme  am stärksten mit stereotypischen bzw. nicht-stereotypischen Aussagen assoziert sind. Zum anderen untersuchen sie welche Ngramme am stärksten mit pro-migrantischen Aussagen, welche Migrant\*innen als Opfer framen, assoziiert sind und welche Ngramme am stärksten mit anti-migrantischen Aussagen, in denen Migrant\*innen als Bedrohung dargestellt werden, assoziiert sind.
Ziel dabei ist, subtilere, nicht-triviale linguistische Muster automatisch zu ermitteln, welche zwar stark mit Stereotypen bzw. einer bestimmten Ausprägung von Stereotypen verbunden (also oft gleichzeitig mit ihnen auftreten), jedoch nicht für uns als solche ersichtlich sind.

Das wollen wir ebenfalls nachbilden.

**Wahrscheinlichkeit von Ngrammen berechnen:**

In [202]:
# Counts of total occurrences of ngrams
def count_ngrams(ngrams, ngram_count_matrx):
    ngram_counts = dict()
    for i, ngram in enumerate(ngrams):
        ngram_counts[ngram] = np.sum(ngram_count_matrx, axis=0)[i]
    return ngram_counts

# Get total number of ngrams
def get_total_count(ngram_counts):
    return sum(ngram_counts.values())

In [203]:
unigram_counts = count_ngrams(unigrams, unigram_count_matrx)
bigram_counts = count_ngrams(bigrams, bigram_count_matrx)
trigram_counts = count_ngrams(trigrams, trigram_count_matrx)

N_unigrams = get_total_count(unigram_counts)
N_bigrams = get_total_count(bigram_counts)
N_trigrams = get_total_count(trigram_counts)

In [204]:
# Compute ngram probability
def get_prob(ngram, ngram_counts, N_ngrams):
    return ngram_counts.get(ngram) / N_ngrams

**Beispiel:** Wahrscheinlichkeit von "terrorismus"

In [205]:
# Probability of occurrence of "terrorismus"
print("Probability of 'terrorismus':")
get_prob("terrorismus", unigram_counts, N_unigrams)

Probability of 'terrorismus':


0.005653021442495127

**Wahrscheinlichkeit der Kookkurrenz von Ngrammen und Labels berechnen:**

In [206]:
# Counts of cooccurrence of ngrams and labels
def count_ngram_label_cooccurrence(ngrams, ngram_count_matrx):
    ngram_label_cooccurrence = dict()
    for i in range(sents.size):
        for j in range(ngrams.size):
            tpl = (ngrams[j], y[i])
            if tpl in ngram_label_cooccurrence.keys():
                ngram_label_cooccurrence[tpl] += ngram_count_matrx[i, j]
            else:    
                ngram_label_cooccurrence[tpl] = ngram_count_matrx[i, j]
    return ngram_label_cooccurrence

In [207]:
unigram_label_coocc = count_ngram_label_cooccurrence(unigrams, unigram_count_matrx)
bigram_label_coocc = count_ngram_label_cooccurrence(bigrams, bigram_count_matrx)
trigram_label_coocc = count_ngram_label_cooccurrence(trigrams, trigram_count_matrx)

In [208]:
# Compute probability of cooccurrence of ngram and label
def get_coocc_prob(ngram, label, ngram_label_coocc, ngram_counts):
    return ngram_label_coocc.get((ngram, label)) / ngram_counts.get(ngram)

**Beispiel:** Wahrscheinlichkeit, dass "terrorismus" mit Label "0" bzw. Label "1" auftritt

In [209]:
# Probability of cooccurrence of "terrorismus" and label 0 (Contains no stereotypes)
print("Probability of 'terrorismus' occurring in sentence with label 0:")
get_coocc_prob("terrorismus", 0, unigram_label_coocc, unigram_counts)

Probability of 'terrorismus' occurring in sentence with label 0:


0.5862068965517241

In [210]:
# Example: Probability of cooccurrence of "terrorismus" and label 1 (Contains stereotypes)
print("Probability of 'terrorismus' occurring in sentence with label 1:")
get_coocc_prob("terrorismus", 1, unigram_label_coocc, unigram_counts)

Probability of 'terrorismus' occurring in sentence with label 1:


0.41379310344827586

**Berechnung der Pointwise Mutual Information (PMI):**

In [211]:
# Compute pmi (pointwise mutual information) of ngrams to each label
def get_ngram_label_pmi(ngram_label_coocc, ngram_counts, ngrams_total):
    """Computes the pmi for an ngram and labels 0 and 1 according to following formula:
    pmi(a, b) = log[P(a,b) / (P(a)*P(b))]
    """
    pmi_0, pmi_1 = dict(), dict()
    # Note: We're assuming that the probability of the labels are both 0.5 (like Sanchez et.al)
    # (like Sanchez et.al)
    p_label = 0.5
    for ngram, label in ngram_label_coocc.keys():
        p_ngram_label_coocc = get_coocc_prob(ngram, label, ngram_label_coocc, ngram_counts)
        p_ngram = get_prob(ngram, ngram_counts, ngrams_total)
        quot = p_ngram_label_coocc / (p_ngram * p_label)
        pmi = 0 if quot == 0 else log(quot)
        if label == 0:
            pmi_0[ngram] = pmi
        else:
            pmi_1[ngram] = pmi
            
    # Sort ngrams according to pmi
    sorted_pmi_0 = sorted(pmi_0.items(), key=lambda x: x[1], reverse=True)
    sorted_pmi_1 = sorted(pmi_1.items(), key=lambda x: x[1], reverse=True)
    
    return sorted_pmi_0, sorted_pmi_1

In [212]:
unigrams_0_pmi, unigrams_1_pmi = get_ngram_label_pmi(unigram_label_coocc, unigram_counts, N_unigrams)
bigrams_0_pmi, bigrams_1_pmi = get_ngram_label_pmi(bigram_label_coocc, bigram_counts, N_bigrams)
trigrams_0_pmi, trigrams_1_pmi = get_ngram_label_pmi(trigram_label_coocc, trigram_counts, N_trigrams)

### Ergebnisse

**Unigramme, die am stärksten mit unstereotypischen Sätzen assoziiert sind:**

In [213]:
print("Unigrams most associated with label 0:")
unigrams_0_pmi[:10]

Unigrams most associated with label 0:


[('abgegeben', 9.23600811872476),
 ('abgegrenzt', 9.23600811872476),
 ('abgelehnt', 9.23600811872476),
 ('abgelehnte', 9.23600811872476),
 ('abgeschlossen', 9.23600811872476),
 ('abgeschlossenen', 9.23600811872476),
 ('abgesetzt', 9.23600811872476),
 ('abgesprochen', 9.23600811872476),
 ('abriegeln', 9.23600811872476),
 ('absatz', 9.23600811872476)]

**Unigramme, die am stärksten mit stereotypischen Sätzen assoziiert sind:**

In [214]:
print("Unigrams most associated with label 1:")
unigrams_1_pmi[:10]

Unigrams most associated with label 1:


[('abgeordneten', 9.23600811872476),
 ('abgestattet', 9.23600811872476),
 ('abgleiten', 9.23600811872476),
 ('abgrenzung', 9.23600811872476),
 ('ablehnen', 9.23600811872476),
 ('abrede', 9.23600811872476),
 ('absage', 9.23600811872476),
 ('abschaffung', 9.23600811872476),
 ('abschlüssen', 9.23600811872476),
 ('absprechen', 9.23600811872476)]

## 2. Aufgabe: Opfer vs. Bedrohung

In [273]:
stereotypes_df = raw_df.loc[raw_df["Frame"] != "x"].copy()
stereotypes_df.loc[stereotypes_df["Frame"] < 3, "Victim"] = 1
stereotypes_df.loc[stereotypes_df["Frame"] > 3, "Victim"] = 0

In [278]:
stereotypes_df.shape

(198, 5)

In [307]:
# Number of sentences where immigrants are framed as threats
sum(stereotypes_df["Victim"] == 0)

54

In [277]:
# Number of sentences in category 3 ("economic resource")
stereotypes_df["Victim"].isnull().sum()

16

In [287]:
# Remove category 3
stereotypes_df.dropna(axis=0, inplace=True)
stereotypes_df.head()

,Term_NDoc_Date,Sentence,Frame,Contains stereotypes,Victim
2,18_100_2015-04-23,"Berlin, Donnerstag, den 23. April 2015 9485 Dr. Dietmar Bartsch (A) (C) (D)(B) bereitstellen – auch in Kenntnis der Tatsache, dass die Übergriffe auf Ausländer, auf Unterkünfte der Asylbewerber zugenommen haben und dass Menschen, die im Ehrenamt diese Willkommenskultur, von der wir alle sprechen, leben, eben nicht wenige Probleme haben.",2.1,1.0,1.0
3,18_100_2015-04-23,"Es ist überhaupt nicht hinnehmbar, wenn der Generalsekretär einer immerhin regierungstragenden Partei, der CSU, sagt, das Asylrecht sei nicht für Sozialtouristen gemacht, die einen Freifahrtschein ins „All-inclusive-Sozialparadies“ buchen wollen.",4.4,1.0,0.0
4,18_100_2015-04-23,"Denn auch unter dem Gesichtspunkt – Kollege Bartsch hat auf die Flüchtlingsströme hingewiesen –, dass wir in den nächsten Jahren große Anstrengungen unternehmen müssen, um Flüchtlinge aufzunehmen und zu integrieren – Integration ist fast genauso wichtig –, muss die Leistungsbereitschaft der Kommunen gewährleistet sein.",4.1,1.0,0.0
11,18_100_2015-04-23,"Die Mehrheit der deutschen Bevölkerung steht beispielhaft zu unserer Willkommenskultur, indem sie vor Ort auf vielfältige Weise die Ankunft und das Leben der Flüchtlinge begleitet und erleichtert.",1.5,1.0,1.0
12,18_100_2015-04-23,"Kolleginnen und Kollegen, das Recht auf Asyl ist und bleibt unantastbar.",1.1,1.0,1.0


In [284]:
raw_sents = stereotypes_df["Sentence"].to_numpy()
sents = apply_preprocessing(raw_sents)
sents.size

182

In [295]:
# Array with original indexes so we can retrieve raw sentences later
orig_indexes = np.arange(raw_sents.size)
orig_indexes.size

182

In [286]:
y = stereotypes_df["Victim"].to_numpy()
y.size

182

In [290]:
# Get ngram counts
unigrams, unigram_count_matrx = get_ngram_count_matrx(sents, 1)
bigrams, bigram_count_matrx = get_ngram_count_matrx(sents, 2)
trigrams, trigram_count_matrx = get_ngram_count_matrx(sents, 3)

unigrams.size, bigrams.size, trigrams.size

(1278, 1844, 1719)

In [292]:
# Get ngram counts with tfidf weights
unigram_tfidf_matrx = transform_to_tfidf(unigram_count_matrx)
bigram_tfidf_matrx = transform_to_tfidf(bigram_count_matrx)
trigram_tfidf_matrx = transform_to_tfidf(trigram_count_matrx)

unigram_tfidf_matrx.shape, bigram_tfidf_matrx.shape, trigram_tfidf_matrx.shape

((182, 1278), (182, 1844), (182, 1719))

In [293]:
# Get sentiment scores
sentiment_scores = get_sentiment(sents).reshape(-1, 1)
sentiment_scores.shape

(182, 1)

#### 1.1. Unigramm-Counts

In [296]:
# Train Support Vector Machine with unigram counts
predictions, results = train_and_evaluate_model(unigram_count_matrx, y, orig_indexes, SVC())
print(results)

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        11
         1.0       0.70      1.00      0.83        26

    accuracy                           0.70        37
   macro avg       0.35      0.50      0.41        37
weighted avg       0.49      0.70      0.58        37



/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

#### 1.2. Bigramm-Counts

In [298]:
# Train Support Vector Machine with bigram counts
predictions, results = train_and_evaluate_model(bigram_count_matrx, y, orig_indexes, SVC())
print(results)

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        11
         1.0       0.70      1.00      0.83        26

    accuracy                           0.70        37
   macro avg       0.35      0.50      0.41        37
weighted avg       0.49      0.70      0.58        37



/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

#### 1.3. Trigramm-Counts

In [299]:
# Train Support Vector Machine with trigram counts
predictions, results = train_and_evaluate_model(trigram_count_matrx, y, orig_indexes, SVC())
print(results)

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        11
         1.0       0.70      1.00      0.83        26

    accuracy                           0.70        37
   macro avg       0.35      0.50      0.41        37
weighted avg       0.49      0.70      0.58        37



/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

#### 2.1. Unigramm-Counts mit Tfidf-Gewichtung

In [300]:
# Train Support Vector Machine with unigram tfidf
predictions, results = train_and_evaluate_model(unigram_tfidf_matrx, y, orig_indexes, SVC())
print(results)

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        11
         1.0       0.70      1.00      0.83        26

    accuracy                           0.70        37
   macro avg       0.35      0.50      0.41        37
weighted avg       0.49      0.70      0.58        37



/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

#### 2.2. Bigramm-Counts mit Tfidf-Gewichtung

In [308]:
# Train Support Vector Machine with bigram tfidf
predictions, results = train_and_evaluate_model(bigram_tfidf_matrx, y, orig_indexes, SVC())
print(results)

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        11
         1.0       0.70      1.00      0.83        26

    accuracy                           0.70        37
   macro avg       0.35      0.50      0.41        37
weighted avg       0.49      0.70      0.58        37



/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

#### 2.3. Trigramm-Counts mit Tfidf-Gewichtung

In [309]:
# Train Support Vector Machine with trigram tfidf
predictions, results = train_and_evaluate_model(trigram_tfidf_matrx, y, orig_indexes, SVC())
print(results)

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        11
         1.0       0.70      1.00      0.83        26

    accuracy                           0.70        37
   macro avg       0.35      0.50      0.41        37
weighted avg       0.49      0.70      0.58        37



/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

### 4. Kombinationen von Features

In [311]:
unigrams_sentiment = combine_features(unigram_count_matrx, sentiment_scores)
bigrams_sentiment = combine_features(bigram_count_matrx, sentiment_scores)
trigrams_sentiment = combine_features(trigram_count_matrx, sentiment_scores)

unigrams_tfidf_sentiment = combine_features(unigram_tfidf_matrx, sentiment_scores)
bigrams_tfidf_sentiment = combine_features(bigram_tfidf_matrx, sentiment_scores)
trigrams_tfidf_sentiment = combine_features(trigram_tfidf_matrx, sentiment_scores)

#### 4.1. Unigramm-Counts + Sentiment

In [316]:
# Train Support Vector Machine with unigrams and sentiment scores
predictions, results = train_and_evaluate_model(unigrams_sentiment, y, orig_indexes, SVC())
print(results)

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        11
         1.0       0.70      1.00      0.83        26

    accuracy                           0.70        37
   macro avg       0.35      0.50      0.41        37
weighted avg       0.49      0.70      0.58        37



/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

#### 4.2. Bigramm-Counts + Sentiment

In [313]:
# Train Support Vector Machine with bigrams and sentiment scores
predictions, results = train_and_evaluate_model(bigrams_sentiment, y, orig_indexes, SVC())
print(results)

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        11
         1.0       0.70      1.00      0.83        26

    accuracy                           0.70        37
   macro avg       0.35      0.50      0.41        37
weighted avg       0.49      0.70      0.58        37



/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

#### 4.3. Trigramm-Counts + Sentiment

In [315]:
# Train Support Vector Machine with trigrams and sentiment scores
predictions, results = train_and_evaluate_model(trigrams_sentiment, y, orig_indexes, SVC())
print(results)

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        11
         1.0       0.70      1.00      0.83        26

    accuracy                           0.70        37
   macro avg       0.35      0.50      0.41        37
weighted avg       0.49      0.70      0.58        37



/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

#### 4.4. Unigramm-Tfidf + Sentiment

In [317]:
# Train Support Vector Machine with unigram tfidf and sentiment scores
predictions, results = train_and_evaluate_model(unigrams_tfidf_sentiment, y, orig_indexes, SVC())
print(results)

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        11
         1.0       0.70      1.00      0.83        26

    accuracy                           0.70        37
   macro avg       0.35      0.50      0.41        37
weighted avg       0.49      0.70      0.58        37



/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

#### 4.5. Bigramm-Tfidf + Sentiment

In [318]:
# Train Support Vector Machine with bigram tfidf and sentiment scores
predictions, results = train_and_evaluate_model(bigrams_tfidf_sentiment, y, orig_indexes, SVC())
print(results)

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        11
         1.0       0.70      1.00      0.83        26

    accuracy                           0.70        37
   macro avg       0.35      0.50      0.41        37
weighted avg       0.49      0.70      0.58        37



/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

#### 4.6. Trigramm-Tfidf + Sentiment

In [319]:
# Train Support Vector Machine with trigram tfidf and sentiment scores
predictions, results = train_and_evaluate_model(trigrams_tfidf_sentiment, y, orig_indexes, SVC())
print(results)

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        11
         1.0       0.70      1.00      0.83        26

    accuracy                           0.70        37
   macro avg       0.35      0.50      0.41        37
weighted avg       0.49      0.70      0.58        37



/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/liane/anaconda3/envs/clt22/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)